<a href="https://colab.research.google.com/github/Sweetydutta8/hello-world/blob/master/fashionmnist%20using%20cnn_kerastuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
print(tf.__version__)

2.2.0


In [7]:
fashion_mnist=keras.datasets.fashion_mnist

In [8]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [10]:
train_images=train_images/255.0
test_images=test_images/255.0

In [11]:
train_images[0].shape

(28, 28)

In [12]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [15]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [17]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.7866 - accuracy: 0.7060 - val_loss: 0.6038 - val_accuracy: 0.7720
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5559 - accuracy: 0.7935 - val_loss: 0.4985 - val_accuracy: 0.8130
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4998 - accuracy: 0.8180 - val_loss: 0.4673 - val_accuracy: 0.8272


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.6865 - accuracy: 0.7466 - val_loss: 0.6322 - val_accuracy: 0.7633
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4928 - accuracy: 0.8215 - val_loss: 0.4592 - val_accuracy: 0.8322
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4391 - accuracy: 0.8421 - val_loss: 0.4159 - val_accuracy: 0.8508


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.7712 - accuracy: 0.7099 - val_loss: 0.5340 - val_accuracy: 0.7980
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5289 - accuracy: 0.8045 - val_loss: 0.4823 - val_accuracy: 0.8173
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4758 - accuracy: 0.8279 - val_loss: 0.4581 - val_accuracy: 0.8340


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 2.3043 - accuracy: 0.1005 - val_loss: 2.3033 - val_accuracy: 0.0973
Epoch 2/3
1688/1688 [==============================] - 9s 6ms/step - loss: 2.3039 - accuracy: 0.0985 - val_loss: 2.3034 - val_accuracy: 0.1003
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3041 - accuracy: 0.1001 - val_loss: 2.3034 - val_accuracy: 0.0985


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 2.3042 - accuracy: 0.1007 - val_loss: 2.3047 - val_accuracy: 0.1027
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 2.3041 - accuracy: 0.0993 - val_loss: 2.3057 - val_accuracy: 0.0942
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 2.3040 - accuracy: 0.0994 - val_loss: 2.3041 - val_accuracy: 0.1032


INFO:tensorflow:Oracle triggered exit


In [18]:
model=tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               4128880   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 4,148,826
Trainable params: 4,148,826
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4047 - accuracy: 0.8548 - val_loss: 0.4309 - val_accuracy: 0.8455
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3781 - accuracy: 0.8623 - val_loss: 0.3756 - val_accuracy: 0.8612
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3592 - accuracy: 0.8686 - val_loss: 0.3776 - val_accuracy: 0.8652
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3403 - accuracy: 0.8761 - val_loss: 0.3653 - val_accuracy: 0.8625
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3244 - accuracy: 0.8826 - val_loss: 0.3913 - val_accuracy: 0.8572
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3062 - accuracy: 0.8879 - val_loss: 0.3433 - val_accuracy: 0.8760
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2893 - accuracy: 0.8937 - val_loss: 0.3246 - val_accuracy